In [179]:
#!/usr/bin/env python
from datetime import datetime
from time import time
from lxml import html,etree
from reviews_final import scrape, write_in_csv
import pandas as pd
import requests,re
import os,sys
import unicodecsv as csv
import argparse

def parse(locality,checkin_date,checkout_date,sort):
    checkIn = checkin_date.strftime("%Y/%m/%d")
    checkOut = checkout_date.strftime("%Y/%m/%d")
    print ("Scraper Inititated for Locality:%s"%locality)
    # TA rendering the autocomplete list using this API
    print ("Finding search result page URL")
    geo_url = 'https://www.tripadvisor.com/TypeAheadJson?action=API&startTime='+str(int(time()))+'&uiOrigin=GEOSCOPE&source=GEOSCOPE&interleaved=true&types=geo,theme_park&neighborhood_geos=true&link_type=hotel&details=true&max=12&injectNeighborhoods=true&query='+locality
    api_response  = requests.get(geo_url, verify=False).json()
    #getting the TA url for th equery from the autocomplete response
    url_from_autocomplete = "http://www.tripadvisor.com"+api_response['results'][0]['url']
    print ('URL found %s'%url_from_autocomplete)
    geo = api_response['results'][0]['value']
    #Formating date for writing to file
    a=url_from_autocomplete
    b=a.split("-")
    s="-"
    c=s.join([b[0],b[1],"oa30",b[2],b[3]])
    d=s.join([b[0],b[1],"oa60",b[2],b[3]])
    urllist = [a,c,d]

    date = checkin_date.strftime("%Y_%m_%d")+"_"+checkout_date.strftime("%Y_%m_%d")
    #form data to get the hotels list from TA for the selected date
    form_data = {'changeSet': 'TRAVEL_INFO',
            'showSnippets': 'false',
            'staydates':date,
            'uguests': '2',
            'sortOrder':sort
        
    }
    
    
    all_hotel=[]
    
    for url_from_autocomplete in urllist:
        print(url_from_autocomplete)

        headers = {
                                'Accept': 'text/javascript, text/html, application/xml, text/xml, */*',
                                'Accept-Encoding': 'gzip,deflate',
                                'Accept-Language': 'en-US,en;q=0.5',
                                'Cache-Control': 'no-cache',
                                'Connection': 'keep-alive',
                                'Content-Type': 'application/x-www-form-urlencoded; charset=utf-8',
                                'Host': 'www.tripadvisor.com',
                                'Pragma': 'no-cache',
                                'Referer': url_from_autocomplete,
                                'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:28.0) Gecko/20100101 Firefox/28.0',
                                'X-Requested-With': 'XMLHttpRequest'
                            }
        cookies=  {"SetCurrency":"USD"}
        print ("Downloading search results page")
        page_response  = requests.post(url = url_from_autocomplete,data=form_data,headers = headers, cookies = cookies, verify=False)
        print ("Parsing results ")
        parser = html.fromstring(page_response.text)
        hotel_lists = parser.xpath('//div[contains(@class,"listItem")]//div[contains(@class,"listing collapsed")]')
        hotel_data = []
        if not hotel_lists:
            hotel_lists = parser.xpath('//div[contains(@class,"listItem")]//div[@class="listing "]')

        for hotel in hotel_lists:
            XPATH_HOTEL_LINK = './/a[contains(@class,"property_title")]/@href'
            XPATH_REVIEWS  = './/a[@class="review_count"]//text()'
            XPATH_RANK = './/div[@class="popRanking"]//text()'
            XPATH_RATING = './/span[contains(@class,"rating")]/@alt' #update this code to get rating
            XPATH_HOTEL_NAME = './/a[contains(@class,"property_title")]//text()'
            XPATH_HOTEL_FEATURES = './/div[contains(@casls,"common_hotel_icons_list")]//li//text()'
            XPATH_HOTEL_PRICE = './/div[contains(@data-sizegroup,"mini-meta-price")]/text()'
            XPATH_VIEW_DEALS = './/div[contains(@data-ajax-preserve,"viewDeals")]//text()'
            XPATH_BOOKING_PROVIDER = './/div[contains(@data-sizegroup,"mini-meta-provider")]//text()'

            raw_booking_provider = hotel.xpath(XPATH_BOOKING_PROVIDER)
            raw_no_of_deals =  hotel.xpath(XPATH_VIEW_DEALS)
            raw_hotel_link = hotel.xpath(XPATH_HOTEL_LINK)
            raw_no_of_reviews = hotel.xpath(XPATH_REVIEWS)
            raw_rank = hotel.xpath(XPATH_RANK)
            raw_rating = hotel.xpath(XPATH_RATING)
            raw_hotel_name = hotel.xpath(XPATH_HOTEL_NAME)
            raw_hotel_features = hotel.xpath(XPATH_HOTEL_FEATURES)
            raw_hotel_price_per_night  = hotel.xpath(XPATH_HOTEL_PRICE)

            url = 'http://www.tripadvisor.com'+raw_hotel_link[0] if raw_hotel_link else  None
            reviews = ''.join(raw_no_of_reviews).replace("reviews","").replace(",","") if raw_no_of_reviews else 0
            rank = ''.join(raw_rank) if raw_rank else None
            rating = ''.join(raw_rating).replace('of 5 bubbles','').strip() if raw_rating else None
            name = ''.join(raw_hotel_name).strip() if raw_hotel_name else None
            hotel_features = ','.join(raw_hotel_features)
            #price_per_night = ''.join(raw_hotel_price_per_night).encode('utf-8').replace('\n','') if raw_hotel_price_per_night else None
            price_per_night = ''.join(raw_hotel_price_per_night).replace('\n','') if raw_hotel_price_per_night else None

            no_of_deals = re.findall("all\s+?(\d+)\s+?",''.join(raw_no_of_deals))
            booking_provider = ''.join(raw_booking_provider).strip() if raw_booking_provider else None

            if no_of_deals:
                no_of_deals = no_of_deals[0]
            else:
                no_of_deals = 0

            data = {
                        'hotel_name':name,
                        'url':url,
                        'locality':locality,
                        'reviews':reviews,
                        'tripadvisor_rating':rating,
                        'checkOut':checkOut,
                        'checkIn':checkIn,
                        'hotel_features':hotel_features,
                        'price_per_night':price_per_night,
                        'no_of_deals':no_of_deals,
                        'booking_provider':booking_provider

            }
            hotel_data.append(data)
            all_hotel.append(data)
    #Referrer is necessary to get the correct response from TA if not provided they will redirect to home page
    my_df=pd.DataFrame(all_hotel)
    print(my_df['hotel_name'])
    return all_hotel
   


In [188]:
data = parse(locality,checkin_date,checkout_date,sort)

Scraper Inititated for Locality:Milan
Finding search result page URL


/Users/hfarooq/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


URL found http://www.tripadvisor.com/Hotels-g187849-Milan_Lombardy-Hotels.html
http://www.tripadvisor.com/Hotels-g187849-Milan_Lombardy-Hotels.html


/Users/hfarooq/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Parsing results 
http://www.tripadvisor.com/Hotels-g187849-oa30-Milan_Lombardy-Hotels.html


/Users/hfarooq/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Parsing results 
http://www.tripadvisor.com/Hotels-g187849-oa60-Milan_Lombardy-Hotels.html


/Users/hfarooq/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Parsing results 
0                                       NYX Hotel Milan
1                         Hilton Garden Inn Milan North
2                                           Hotel Berna
3                                Eco-Hotel La Residenza
4                                 Grand Visconti Palace
5                             Radisson Blu Hotel, Milan
6                              Klima Hotel Milano Fiere
7                                      43 Station Hotel
8                        Best Western Plus Hotel Galles
9                               The Square Milano Duomo
10                                   ibis Milano Centro
11                                     Enterprise Hotel
12                                       Hotel Da Vinci
13                                        Hotel Bernina
14                       B&B Hotel Milano Sant'Ambrogio
15                                   Hotel Sanpi Milano
16                             UNAHOTELS Century Milano
17                             

In [189]:
len(data)

92

In [132]:
my_df = pd.DataFrame(data)

In [134]:
print(my_df['hotel_name'])

0                 Radisson Blu Hotel, Milan
1                           Hotel VIU Milan
2                     Hotel Milano Castello
3                        Santa Marta Suites
4                               Hotel Manin
5     Best Western Plus Hotel Felice Casati
6                             Hotel Bernina
7                             Uptown Palace
8        Rosa Grand – Starhotels Collezione
9                  Hotel Principe Di Savoia
10                                 44 Suite
11               Best Western Hotel Madison
12                         Park Hyatt Milan
13                            The Hub Hotel
14                          NU Hotel Milano
15                               Lido Hotel
16                Duomo - Apartments Milano
17                   Hotel degli Arcimboldi
18                            Ca' Monteggia
19                      Armani Hotel Milano
20                  Hotel RossoVino - Milan
21                      Galleria Vik Milano
22                           Hot

In [126]:
for i in data:
    for y in i:
        print (y,':',i[y])

hotel_name : Radisson Blu Hotel, Milan
url : http://www.tripadvisor.com/Hotel_Review-g187849-d646332-Reviews-Radisson_Blu_Hotel_Milan-Milan_Lombardy.html
locality : Milan
reviews : 2390 
tripadvisor_rating : None
checkOut : 2019/11/05
checkIn : 2019/11/01
hotel_features : 
price_per_night : $106
no_of_deals : 11
booking_provider : RadissonHotels.com
hotel_name : Hotel VIU Milan
url : http://www.tripadvisor.com/Hotel_Review-g187849-d12199274-Reviews-Hotel_VIU_Milan-Milan_Lombardy.html
locality : Milan
reviews : 291 
tripadvisor_rating : None
checkOut : 2019/11/05
checkIn : 2019/11/01
hotel_features : 
price_per_night : $231
no_of_deals : 18
booking_provider : Booking.com
hotel_name : Hotel Milano Castello
url : http://www.tripadvisor.com/Hotel_Review-g187849-d13237496-Reviews-Hotel_Milano_Castello-Milan_Lombardy.html
locality : Milan
reviews : 160 
tripadvisor_rating : None
checkOut : 2019/11/05
checkIn : 2019/11/01
hotel_features : 
price_per_night : $212
no_of_deals : 11
booking_provi

In [ ]:
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Paris"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Chicago"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "New York"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "San Francisco"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Istanbul"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Singapore"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Bombay"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Bangkok"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Sydney"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "London"

In [ ]:
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Rome"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Athens"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Miami"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Austin"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Tokyo"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Bali"

In [ ]:
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Osaka"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Hong Kong"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Dubai"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Phuket"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Seoul"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Prague"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Milan"

In [66]:
test="2019/11/01,2019/11/05,popularity,Milan"

In [67]:
parser = argparse.ArgumentParser()
parser.add_argument('checkin_date',help = 'Hotel Check In Date (Format: YYYY/MM/DD')
sortorder_help = """
    available sort orders are :\n
    priceLow - hotels with lowest price,
    distLow : Hotels located near to the search center,
    recommended: highest rated hotels based on traveler reviews,
    popularity :Most popular hotels as chosen by Tipadvisor users
    """
parser.add_argument('checkout_date',help = 'Hotel Chek Out Date (Format: YYYY/MM/DD)')
parser.add_argument('sort',help = sortorder_help,default ='popularity ')
parser.add_argument('locality',help = 'Search Locality')
print(test)

2019/11/01,2019/11/05,popularity,Milan


In [68]:
test.split(',')

['2019/11/01', '2019/11/05', 'popularity', 'Milan']

In [88]:
args = parser.parse_args(test.split(','))
print (args)
locality = args.locality
checkin_date = datetime.strptime(args.checkin_date,"%Y/%m/%d")
checkout_date = datetime.strptime(args.checkout_date,"%Y/%m/%d")
sort= args.sort
checkIn = checkin_date.strftime("%Y/%m/%d")
checkOut = checkout_date.strftime("%Y/%m/%d")
today = datetime.now()

Namespace(checkin_date='2019/11/01', checkout_date='2019/11/05', locality='Milan', sort='popularity')


In [87]:
args
print(locality)

Milan


In [185]:
if today<datetime.strptime(checkIn,"%Y/%m/%d") and datetime.strptime(checkIn,"%Y/%m/%d")<datetime.strptime(checkOut,"%Y/%m/%d"):
        data = parse(locality,checkin_date,checkout_date,sort)
        print ("Writing to output file tripadvisor_data.csv")
        with open('tripadvisor_data.csv','wb')as csvfile:
            fieldnames = ['hotel_name','url','locality','reviews','tripadvisor_rating','checkIn','checkOut','price_per_night','booking_provider','no_of_deals','hotel_features']
            my_df  = pd.DataFrame(columns = fieldnames)
            for row in  data:
                my_df.loc[len(my_df)] = [row['hotel_name'],row['url'],row['locality'],row['reviews'],row['tripadvisor_rating'],row['checkIn'],row['checkOut'],row['price_per_night'],row['booking_provider'],row['no_of_deals'],row['hotel_features']]
            DB_COLUMN   = 'review_body'
            DB_COLUMN1 = 'review_date'
            DB_COLUMN2 = 'hotelName'
            DB_COLUMN3 = 'hotelUrl'
            start_urls = []
            for index, row2 in my_df.iterrows():
                start_urls.append(row2['url'])
            lang = 'en'
    
            headers = [
            DB_COLUMN,
            DB_COLUMN1,
            DB_COLUMN2,
            DB_COLUMN3
            ]
            reviews_df  = pd.DataFrame(columns = headers)
            
            for url in start_urls:

                # get all reviews for 'url' and 'lang'
                items = scrape(url, lang)
                filename = 'hotelReviewsIn' + locality + '__' + lang
                filename2 = "HotelListIn" + locality + '__' + lang
                if not items:
                    print('No reviews')
                else:
                    # write in CSV
                    for i in items:
                        reviews_df.loc[len(reviews_df)] = [i['review_body'],i['review_date'],my_df.loc[my_df['url'] == url]['hotel_name'], url]
            ci = checkIn.replace("/", "")
            co = checkOut.replace("/", "")
            reviews_df.to_csv(filename + ci +co+'.csv', encoding='utf-8')     
            my_df.to_csv(filename2 + ci +co+'.csv', encoding='utf-8')       
    #checking whether the entered date is already passed
elif today>datetime.strptime(checkIn,"%Y/%m/%d") or today>datetime.strptime(checkOut,"%Y/%m/%d"):
     print ("Invalid Checkin date: Please enter a valid checkin and checkout dates,entered date is already passed")

elif datetime.strptime(checkIn,"%Y/%m/%d")>datetime.strptime(checkOut,"%Y/%m/%d"):
     print ("Invalid Checkin date: CheckIn date must be less than checkOut date")


Scraper Inititated for Locality:Milan
Finding search result page URL


/Users/hfarooq/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


URL found http://www.tripadvisor.com/Hotels-g187849-Milan_Lombardy-Hotels.html
http://www.tripadvisor.com/Hotels-g187849-Milan_Lombardy-Hotels.html


/Users/hfarooq/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Parsing results 
http://www.tripadvisor.com/Hotels-g187849-oa30-Milan_Lombardy-Hotels.html


...

/Users/hfarooq/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Parsing results 
0                            Hotel Bernina
1           NH Collection Milano President
2                       Hotel Milano Scala
3                       Starhotels E.c.ho.
4                             Hilton Milan
5                          Hotel VIU Milan
6                         Hotel Rex Milano
7                      Hotel Dei Cavalieri
8                              nhow Milano
9                            Biocity Hotel
10    Four Points by Sheraton Milan Center
11                    Antares Hotel Rubens
12                         Chateau Monfort
13                         iQ Hotel Milano
14                               EuroHotel
15               NH Milano Palazzo Moscova
16                 UNAHOTELS Cusani Milano
17              Starhotels Business Palace
18      B&B Hotel Milano Cenisio Garibaldi
19                            Hotel Silver
20                             Acca Palace
21                    Best Western Antares
22        B&B Hotel Milano Central St

[get_reviews_ids] data-reviewid: ['675166245', '673145606', '672202076']

--- review ---

  review_body : Small but quaint little hotel tucked away minutes from grand central station. Great staff, cute little dining area with a delicious breakfast of various pastries. Single room quite small but well equipped.
  review_date : May 20, 2019

--- review ---

  review_body : A really pleasant spot, excellent location, clean and modern throughout with thoughtful design in room with limited space. Good choices for breakfast and pleasant and helpful reception and dining-room staff.
  review_date : May 10, 2019

--- review ---

  review_body : Very comfortable beds, newly renovated and quiet rooms in a good location near Central station; the rooms are not big but as they are really reasonably priced, this is fine. The breakfast buffet is plentiful and served in a nice room, also newly renovated
  review_date : May 7, 2019

[parse_reviews] url: http://www.tripadvisor.com/Hotel_Review-g187849-d2

[get_reviews_ids] data-reviewid: ['614360345', '611108895', '608654850']

--- review ---

  review_body : Good location and the personnel was very friendly, very good amenities and the breakfast Room was very clean and spacious, also the Food was very good. The Room was very clean all the times. I will recommed this Hotel to my friends and relatives. Also the location of this Hotel is ve4y good because is very close to the Milano Centrale were you can take the Trains to go any place you want.
  review_date : September 6, 2018

--- review ---

  review_body : Within in walking distance to the train station, the Bernina is an ideal spot for an affordable stay. While the rooms are small, the hotel is quiet and the service is very good. Our included breakfast was fantastic. There are restaurants up and down the street and the neighborhood is safe at any hour. This is a great stay.
  review_date : August 27, 2018

--- review ---

  review_body : Everyone was friendly and I loved their break

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [98]:
data = parse(locality,checkin_date,checkout_date,sort)

Scraper Inititated for Locality:Milan
Finding search result page URL


/Users/hfarooq/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


URL found http://www.tripadvisor.com/Hotels-g187849-Milan_Lombardy-Hotels.html
http://www.tripadvisor.com/Hotels-g187849-Milan_Lombardy-Hotels.html


/Users/hfarooq/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Parsing results 
http://www.tripadvisor.com/Hotels-g187849-oa30-Milan_Lombardy-Hotels.html


/Users/hfarooq/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Parsing results 
http://www.tripadvisor.com/Hotels-g187849-oa60-Milan_Lombardy-Hotels.html


/Users/hfarooq/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Parsing results 


In [99]:
len(data)

1

In [ ]:
http://www.tripadvisor.com/Hotels-g187849-Milan_Lombardy-Hotels.html
https://www.tripadvisor.com/Hotels-g187849-oa30-Milan_Lombardy-Hotels.html
https://www.tripadvisor.com/Hotels-g187849-oa60-Milan_Lombardy-Hotels.html

In [45]:
a="http://www.tripadvisor.com/Hotels-g187849-Milan_Lombardy-Hotels.html"

In [48]:
b=a.split("-")

In [49]:
print (b)

['http://www.tripadvisor.com/Hotels', 'g187849', 'Milan_Lombardy', 'Hotels.html']


In [50]:
print(b[0])

http://www.tripadvisor.com/Hotels


In [51]:
s="-"

In [62]:
c=s.join([b[0],b[1],"oa30",b[2],b[3]])
d=s.join([b[0],b[1],"oa60",b[2],b[3]])

In [63]:
urllist = [a,c,d]

In [64]:
urllist

['http://www.tripadvisor.com/Hotels-g187849-Milan_Lombardy-Hotels.html',
 'http://www.tripadvisor.com/Hotels-g187849-oa30-Milan_Lombardy-Hotels.html',
 'http://www.tripadvisor.com/Hotels-g187849-oa60-Milan_Lombardy-Hotels.html']